# BUILDING OUR DATAFRAME -- notebook 2

This notebook carry out the remaining cleaning process and the remaining analysis part is on the notebook 3.

Author: Xuyuan Zhang; Date: Feb. 15. 2024

## we can try to map it here. 

Instead of visualizing this image right away, it is recommended to mask our region of interest as it will save computational power. Make sure the coordinate reference system (CRS) of the GeoJSON file is converted to the CRS of the RGB image created above.

We now construct our interesting area:

In [1]:
import arcpy
import numpy as np
import os
import glob

In [2]:
os.chdir(r'E:\umich\final project\data')

In [3]:

input_rasters = ['S2B_MSIL2A_20230608T161829_N0509_R040_T16TGN_20230608T192144.SAFE.tiff',
                      'S2B_MSIL2A_20230608T161829_N0509_R040_T17TKG_20230608T192144.SAFE.tiff',
                      'S2B_MSIL2A_20230618T161829_N0509_R040_T17TLG_20230618T191812.SAFE.tiff',
                      'S2B_MSIL2A_20230618T161829_N0509_R040_T17TLH_20230618T191812.SAFE.tiff']
output_directory = os.getcwd() # since we already changed the drectory to where the data is.
output_raster_name = "mosaic.tif"  # Name of the output mosaic raster

# Specify the coordinate system using EPSG code
coordinate_system = arcpy.SpatialReference(4326)  # EPSG code for WGS 1984

# Updated parameters based on your specifications
pixel_type = "16_BIT_UNSIGNED"  # For uint16 data type
number_of_bands = 4  # For three bands

# Optional parameter (customize as needed)
mosaic_method = "LAST"  # Determines how the pixels from overlapping areas will be resolved

# Execute MosaicToNewRaster using arcpy.management
arcpy.management.MosaicToNewRaster(
    input_rasters=input_rasters,
    output_location = output_directory,
    raster_dataset_name_with_extension=output_raster_name,
    coordinate_system_for_the_raster=coordinate_system,
    pixel_type=pixel_type,
    number_of_bands=number_of_bands,
    mosaic_method=mosaic_method
)

print("Mosaic to new raster completed successfully.")


Mosaic to new raster completed successfully.


## Extract the given region

To speed up the calculation because our final result will be pretty large, we have to clip the raster to small areas so that we can execute the deep learning extraction faster

In [4]:
# Assuming `input_rasters` contains paths to the TIFF files you want to clip
output_directory = os.getcwd()
cleaned_input = "mosaic.tif"

clipped_raster_path = f"{output_directory}\\{cleaned_input.replace('.tif', '_clipped.tif')}"


you can revise the geodatabse to make it your own

In [6]:
# Define the path where the geodatabase should be created (adjust the path as needed)
gdb_folder_path = "../mask area"  # Folder where the GDB should be created
gdb_name = "michigan.gdb"  # Name of the geodatabase

# Create the geodatabase
arcpy.CreateFileGDB_management(gdb_folder_path, gdb_name)

geodatabase_path = f"{gdb_folder_path}/{gdb_name}"


In [7]:
# Path to the GeoJSON file
geojson_path = "../mask area/detroit.json"
feature_class_name = 'Detroit_Area'

# Convert GeoJSON to a feature class in a geodatabase
arcpy.conversion.JSONToFeatures(geojson_path, f"{geodatabase_path}/{feature_class_name}")

<Result '..\\mask area\\michigan.gdb\\Detroit_Area'>

In [8]:
# Perform the clipping
arcpy.management.Clip(
    in_raster=cleaned_input, 
    out_raster=clipped_raster_path, 
    in_template_dataset=f"{geodatabase_path}/{feature_class_name}",
    clipping_geometry="ClippingGeometry",
    maintain_clipping_extent="NO_MAINTAIN_EXTENT"
)


<Result 'E:\\umich\\final project\\data\\mosaic_clipped.tif'>

potential reference:

https://old-monk.medium.com/a-precise-guide-to-investigate-sentinel-2-data-in-python-downloading-visualizing-2cc8d9d60021

https://ieeexplore.ieee.org/document/10144166